Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menu bar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menu bar, select Cell$\rightarrow$Run All).

Below, please fill in your name and collaborators:

In [ ]:
NAME = "Group 35"
COLLABORATORS = ""

In [ ]:
# Group Project - Dataset Analysis and Interpretation
**(30 points total)**

## Assignment tasks:

The objective is to select a data set, transform and clean it, conduct a data analysis, and report key 
statistical features and insights of the data. If you are already familiar with regression and/or classification 
and would like to try out one of the predictive modelling  algorithms in scikit-learn it is fine to do so 
but this is not required.



Hypothesis statement: 
Which attributes make a team more likely to win the game? 
Predictors include ( but are not limited to): 
    Shots taken, goals scored in first period, face off wins, head coach, home arena.



## Downloading NHL data

In [1]:
# all imports and env variables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime
import pandas_datareader.data as web
from pandas import DataFrame, Series
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import scipy, scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

game_id   Game key field as assigned by the NHL

season type:  R=Regular Season, P=Playoff

date_time:date_time_GMT

away_team_id: Not strictly rational as can be calculated form game_team_stats, provided here as a convenience

home_team_id: Not strictly rational as can be calculated form game_team_stats, provided here as a convenience

away_goals: Not strictly rational as can be calculated form game_team_stats, provided here as a convenience

home_goals: Not strictly rational as can be calculated form game_team_stats, provided here as a convenience

outcome home_rink_side_start Indicates the direction of play relative to the Time/Score keepers. 
XY coordinates are relative to this.

venue: venue_link

venue_time_zone_id

venue_time_zone_offset

venue_time_zone_tz

In [2]:
# Import the data set
game = pd.read_csv('game.csv')
game.head()

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT


In [4]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11434 entries, 0 to 11433
Data columns (total 16 columns):
game_id                   11434 non-null int64
season                    11434 non-null int64
type                      11434 non-null object
date_time                 11434 non-null object
date_time_GMT             11434 non-null object
away_team_id              11434 non-null int64
home_team_id              11434 non-null int64
away_goals                11434 non-null int64
home_goals                11434 non-null int64
outcome                   11434 non-null object
home_rink_side_start      11140 non-null object
venue                     11434 non-null object
venue_link                11434 non-null object
venue_time_zone_id        11434 non-null object
venue_time_zone_offset    11434 non-null int64
venue_time_zone_tz        11434 non-null object
dtypes: int64(7), object(9)
memory usage: 1.4+ MB


In [5]:
#checking for total null values
game.isnull().sum()

game_id                     0
season                      0
type                        0
date_time                   0
date_time_GMT               0
away_team_id                0
home_team_id                0
away_goals                  0
home_goals                  0
outcome                     0
home_rink_side_start      294
venue                       0
venue_link                  0
venue_time_zone_id          0
venue_time_zone_offset      0
venue_time_zone_tz          0
dtype: int64

In [25]:
# Analyze the missing values
game.describe(include=[np.object])

,type,date_time,date_time_GMT,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_tz
count,11434,11434,11434,11434,11140,11434,11434,11434,11434
unique,2,1923,7156,6,2,82,29,11,8
top,R,2012-03-11,2015-11-15T00:00:00Z,home win REG,right,TD Garden,/api/v1/venues/null,America/New_York,EDT
freq,10642,16,9,4857,5958,408,10319,4974,5549


In [6]:
# Import the data set
game_team_stats = pd.read_csv('game_teams_stats.csv')
game_team_stats

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,1,away,False,OT,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
1,2011030221,4,home,True,OT,Peter Laviolette,4,36,27,6,6,1,55.1,13,4
2,2011030222,1,away,True,REG,Peter DeBoer,4,35,32,12,4,0,50.9,8,7
3,2011030222,4,home,False,REG,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,4,away,False,OT,Peter Laviolette,3,28,28,4,5,1,50.8,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22863,2018030415,6,home,False,REG,Bruce Cassidy,1,39,43,2,3,0,40.6,4,11
22864,2018030416,6,away,True,REG,Bruce Cassidy,5,32,27,10,4,1,41.3,4,10
22865,2018030416,19,home,False,REG,Craig Berube,1,29,29,20,4,0,58.7,12,11
22866,2018030417,19,away,True,REG,Craig Berube,4,20,36,2,0,0,49.0,7,8


In [10]:
game_team_stats.isnull().sum()

game_id                   0
team_id                   0
HoA                       0
won                       0
settled_in                0
head_coach                0
goals                     0
shots                     0
hits                      0
pim                       0
powerPlayOpportunities    0
powerPlayGoals            0
faceOffWinPercentage      0
giveaways                 0
takeaways                 0
dtype: int64

In [8]:
game_team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22868 entries, 0 to 22867
Data columns (total 15 columns):
game_id                   22868 non-null int64
team_id                   22868 non-null int64
HoA                       22868 non-null object
won                       22868 non-null bool
settled_in                22868 non-null object
head_coach                22868 non-null object
goals                     22868 non-null int64
shots                     22868 non-null int64
hits                      22868 non-null int64
pim                       22868 non-null int64
powerPlayOpportunities    22868 non-null int64
powerPlayGoals            22868 non-null int64
faceOffWinPercentage      22868 non-null float64
giveaways                 22868 non-null int64
takeaways                 22868 non-null int64
dtypes: bool(1), float64(1), int64(10), object(3)
memory usage: 2.5+ MB


In [ ]:
# Analyze the missing values
Titanic.describe(include=[np.object])

In [24]:
#Look at win rate rate by goals
game_team_stats.groupby(['goals'])['won'].count()


goals
0     1531
1     4083
2     5316
3     4894
4     3718
5     2050
6      852
7      324
8       77
9       17
10       6
Name: won, dtype: int64

In [21]:
#Look at win rate rate by goals
game_team_stats.groupby(['goals','shots'])['won'].aggregate('count')

goals  shots
0      10        1
       12        2
       13        7
       14        8
       15       11
                ..
10     34        1
       36        2
       40        1
       44        1
       47        1
Name: won, Length: 393, dtype: int64

In [42]:
game_team_stats.pivot_table('won', index='goals', columns=['shots'])

shots,10,11,12,13,14,15,16,17,18,19,...,54,55,56,57,58,59,60,62,63,64
goals,,,,,,,,,,,,,,,,,,,,,
0,0.0,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,...,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.2,0.294118,0.100000,0.166667,0.102564,0.228070,0.179487,...,0.000000,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,0.0,1.0,0.0,0.6,0.545455,0.454545,0.500000,0.404762,0.428571,0.375000,...,0.333333,NaN,0.5,NaN,NaN,NaN,1.0,0.0,NaN,NaN
3,NaN,NaN,1.0,1.0,0.923077,0.714286,0.666667,0.709677,0.698113,0.710145,...,0.000000,0.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,0.875000,0.750000,0.777778,0.933333,0.904762,0.928571,...,NaN,1.0,0.5,1.0,0.333333,NaN,NaN,NaN,1.0,1.0
5,NaN,1.0,1.0,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,0.846154,...,NaN,0.5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,1.000000,1.000000,...,1.000000,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
game_team_stats.pivot_table('won', index='goals', columns=['shots', 'pim'])

shots   10             11                       12       ...   58            \
pim     0    6    19   0    6    10   11   13   6    8   ...   10   13   15   
goals                                                    ...                  
0      NaN  NaN  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
1      0.0  NaN  NaN  0.0  0.0  0.0  NaN  NaN  NaN  0.0  ...  NaN  NaN  NaN   
2      NaN  0.0  NaN  NaN  NaN  NaN  NaN  1.0  NaN  0.0  ...  NaN  NaN  NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  ...  NaN  NaN  NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.0  1.0  0.0   
5      NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN  1.0  NaN  ...  NaN  NaN  NaN   
6      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
7      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
8      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
9      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
10     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

shots   59        60        62   63   64  
pim     4    11   4    14   10   6    23  
goals                                     
0      NaN  0.0  NaN  NaN  NaN  NaN  NaN  
1      0.0  NaN  NaN  NaN  NaN  NaN  NaN  
2      NaN  NaN  NaN  1.0  0.0  NaN  NaN  
3      NaN  NaN  0.0  NaN  NaN  NaN  NaN  
4      NaN  NaN  NaN  NaN  NaN  1.0  1.0  
5      NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6      NaN  NaN  NaN  NaN  NaN  NaN  NaN  
7      NaN  NaN  NaN  NaN  NaN  NaN  NaN  
8      NaN  NaN  NaN  NaN  NaN  NaN  NaN  
9      NaN  NaN  NaN  NaN  NaN  NaN  NaN  
10     NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[11 rows x 1415 columns]

In [31]:
game.groupby(['type'])['outcome'].aggregate('count')

type
P      792
R    10642
Name: outcome, dtype: int64

In [32]:
game.groupby(['outcome'])['type'].aggregate('count')

outcome
away win OT      734
away win REG    3881
away win SO      591
home win OT      791
home win REG    4857
home win SO      580
Name: type, dtype: int64

In [36]:
game_stats = pd.merge(game, game_team_stats, on = 'game_id')
game_stats

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,...,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,...,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
1,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,...,Peter Laviolette,4,36,27,6,6,1,55.1,13,4
2,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,...,Peter DeBoer,4,35,32,12,4,0,50.9,8,7
3,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,...,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,...,Peter Laviolette,3,28,28,4,5,1,50.8,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22863,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,...,Bruce Cassidy,1,39,43,2,3,0,40.6,4,11
22864,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,...,Bruce Cassidy,5,32,27,10,4,1,41.3,4,10
22865,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,...,Craig Berube,1,29,29,20,4,0,58.7,12,11
22866,2018030417,20182019,P,2019-06-13,2019-06-13T00:00:00Z,19,6,4,1,away win REG,...,Craig Berube,4,20,36,2,0,0,49.0,7,8


In [37]:
game_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22868 entries, 0 to 22867
Data columns (total 30 columns):
game_id                   22868 non-null int64
season                    22868 non-null int64
type                      22868 non-null object
date_time                 22868 non-null object
date_time_GMT             22868 non-null object
away_team_id              22868 non-null int64
home_team_id              22868 non-null int64
away_goals                22868 non-null int64
home_goals                22868 non-null int64
outcome                   22868 non-null object
home_rink_side_start      22280 non-null object
venue                     22868 non-null object
venue_link                22868 non-null object
venue_time_zone_id        22868 non-null object
venue_time_zone_offset    22868 non-null int64
venue_time_zone_tz        22868 non-null object
team_id                   22868 non-null int64
HoA                       22868 non-null object
won                       22868 non

In [57]:
game_stats.groupby(['won'])['pim'].mean()
#double count due to the game_team_stats containing both away and home team status 

won
False    10.487668
True      9.863915
Name: pim, dtype: float64

In [58]:
game_stats.groupby(['won'])['hits'].mean()

won
False    24.184100
True     23.031048
Name: hits, dtype: float64

In [67]:
game_stats.groupby(['won'])['hits', 'giveaways', 'takeaways', 'goals','faceOffWinPercentage', 'pim', 'powerPlayGoals', 'shots'].mean()

,hits,giveaways,takeaways,goals,faceOffWinPercentage,pim,powerPlayGoals,shots
won,,,,,,,,
False,24.184100,8.709900,6.507084,1.767535,49.752746,10.487668,0.419188,30.361991
True,23.031048,8.586409,7.332517,3.718559,50.247254,9.863915,0.765524,30.430820


In [13]:
#aWNTimePlot = pd.plotting.autocorrelation_plot(df_ibm_adj_close)
#aWNTimePlot.set_title("An Autocorrelation Graph of IBM Month End Adj Close Stock")
#aWNTimePlot.set_ylabel("Correlation")
#aWNTimePlot.set_xlabel("Lag (Months)")
